In [1]:
# ===========================
# Install & environment
# ===========================
!pip -q install monai nibabel scikit-learn timm einops tqdm

import os
os.environ["PYTORCH_DISABLE_DYNAMO"] = "1"
os.environ["TORCHDYNAMO_DISABLE"] = "1"

# ===========================
# Imports
# ===========================
import torch, numpy as np, random, math, warnings, json
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

from monai.data import Dataset
from monai import transforms as T
from monai.config import print_config

import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from collections import defaultdict, Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix

warnings.filterwarnings("ignore")

# ===========================
# Repro, device, paths
# ===========================
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DRIVE = Path("/content/drive/MyDrive")
DATA_ROOT = DRIVE / "OASIS3_Preprocessed_Data"   # expects AD/ and CN/
SAVE_DIR  = DRIVE / "OASIS3_Preprocessed_Data"
SAVE_DIR.mkdir(parents=True, exist_ok=True)

print("Torch:", torch.__version__, "| Device:", device)
print_config()
print("Data root:", DATA_ROOT)

# ====================
# CONFIG (tune here)
# ====================
# Phase-1 (baseline) — stays accuracy-friendly but still robust
DROPOUT_P1      = 0.00
LR_PHASE1       = 5e-5
EPOCHS_PHASE1   = 80
PATIENCE_P1     = 12
USE_CLASS_WEIGHT_P1 = True      # pos_weight for AD to help balance
MIXUP_ALPHA_P1  = 0.20          # mild mixup in P1

# Phase-2 (fine-tune) — *gentler* to recover overall accuracy while keeping BA
DROPOUT_P2      = 0.10
LR_PHASE2       = 5e-6          # smaller LR for stability
EPOCHS_PHASE2   = 20
PATIENCE_P2     = 8
USE_CLASS_WEIGHT_P2 = False     # turn OFF class weight in FT (improves accuracy)
MIXUP_ALPHA_P2  = 0.00          # turn OFF mixup in FT

# Optional: freeze backbone in fine-tune (only train classifier head)
FREEZE_BACKBONE_IN_FT = False   # set True to try even stabler FT (often +acc)

# Loss asymmetry (for Phase-1 when USE_CLASS_WEIGHT=True)
POS_WEIGHT_POWER = 1.25         # pos_weight = (n_cn/n_ad)**power

# Batching / data
BATCH_SIZE   = 4
NUM_WORKERS  = 2
PIN_MEMORY   = True
GRAD_ACCUM_TARGET = 8

# Split
TEST_FRACTION   = 0.15
VAL_FRACTION_OF_TRAINVAL = 0.20

# ===========================
# Data discovery
# ===========================
CLASS_TO_IDX = {"AD": 0, "CN": 1}
IDX_TO_CLASS = {v:k for k,v in CLASS_TO_IDX.items()}

def list_nii(d: Path):
    return sorted([p for p in d.rglob("*") if p.is_file() and p.name.lower().endswith((".nii",".nii.gz"))])

ad_files = list_nii(DATA_ROOT / "AD")
cn_files = list_nii(DATA_ROOT / "CN")
items = [{"image": str(p), "label": CLASS_TO_IDX["AD"]} for p in ad_files] + \
        [{"image": str(p), "label": CLASS_TO_IDX["CN"]} for p in cn_files]
assert len(items) > 0, "No NIfTI files found. Check /MyDrive/OASIS3_Preprocessed_Data/AD and /CN."

labels_all = np.array([it["label"] for it in items])
print(f"Found | AD={len(ad_files)} | CN={len(cn_files)} | Total={len(items)}")
u, c = np.unique(labels_all, return_counts=True)
print("Class counts:", {IDX_TO_CLASS[int(ui)]: int(ci) for ui,ci in zip(u,c)})

# ==================
# Transforms
# ==================
TARGET_SPACING = (1.0, 1.0, 1.0)
TARGET_SIZE    = (128, 160, 160)

train_tf = T.Compose([
    T.LoadImaged(keys=["image"]),
    T.EnsureChannelFirstd(keys=["image"]),
    T.Orientationd(keys=["image"], axcodes="RAS"),
    T.Spacingd(keys=["image"], pixdim=TARGET_SPACING, mode=("bilinear")),
    T.ScaleIntensityRangePercentilesd(keys=["image"], lower=0.5, upper=99.5, b_min=0.0, b_max=1.0, clip=True),
    T.CropForegroundd(keys=["image"], source_key="image"),
    T.ResizeWithPadOrCropd(keys=["image"], spatial_size=TARGET_SIZE),
    # mild augs (safe globally)
    T.RandFlipd(keys=["image"], prob=0.2, spatial_axis=[0]),
    T.RandFlipd(keys=["image"], prob=0.2, spatial_axis=[1]),
    T.RandFlipd(keys=["image"], prob=0.2, spatial_axis=[2]),
    T.RandAffined(keys=["image"], prob=0.10,
                  rotate_range=(np.deg2rad(8),)*3,
                  scale_range=(0.08,0.08,0.08),
                  translate_range=(4,4,4),
                  mode=("bilinear"), padding_mode="border"),
    T.EnsureTyped(keys=["image"], dtype=torch.float32),
    T.ToTensord(keys=["image"]),
])

val_tf = T.Compose([
    T.LoadImaged(keys=["image"]),
    T.EnsureChannelFirstd(keys=["image"]),
    T.Orientationd(keys=["image"], axcodes="RAS"),
    T.Spacingd(keys=["image"], pixdim=TARGET_SPACING, mode=("bilinear")),
    T.ScaleIntensityRangePercentilesd(keys=["image"], lower=0.5, upper=99.5, b_min=0.0, b_max=1.0, clip=True),
    T.CropForegroundd(keys=["image"], source_key="image"),
    T.ResizeWithPadOrCropd(keys=["image"], spatial_size=TARGET_SIZE),
    T.EnsureTyped(keys=["image"], dtype=torch.float32),
    T.ToTensord(keys=["image"]),
])

from torch.utils.data import Sampler, DataLoader

class BalancedBinaryBatchSampler(Sampler):
    """Balanced mini-batches over LOCAL indices: 50% AD (0), 50% CN (1)."""
    def __init__(self, indices_local, labels_local, batch_size):
        assert batch_size % 2 == 0
        self.batch_size = batch_size
        self.half = batch_size // 2
        idx_by_class = defaultdict(list)
        for i in indices_local:
            idx_by_class[int(labels_local[i])].append(i)
        self.idx0 = idx_by_class[0][:]
        self.idx1 = idx_by_class[1][:]
        if len(self.idx0)==0 or len(self.idx1)==0:
            raise ValueError("Both classes needed in the TRAIN split.")
        self.num_batches = math.ceil(max(len(self.idx0), len(self.idx1))*2 / self.batch_size)

    def __len__(self): return self.num_batches
    def __iter__(self):
        for _ in range(self.num_batches):
            b0 = [random.choice(self.idx0) for _ in range(self.half)]
            b1 = [random.choice(self.idx1) for _ in range(self.half)]
            batch = b0 + b1
            random.shuffle(batch)
            yield batch

def build_balanced_loaders_from_indices(idx_train_global, idx_val_global, batch_size=BATCH_SIZE):
    train_items_loc = [items[i] for i in idx_train_global]
    val_items_loc   = [items[i] for i in idx_val_global]
    train_ds_loc = Dataset(train_items_loc, transform=train_tf)
    val_ds_loc   = Dataset(val_items_loc,   transform=val_tf)

    train_local_indices = np.arange(len(train_items_loc))
    train_labels_local  = np.array([it["label"] for it in train_items_loc])
    train_sampler = BalancedBinaryBatchSampler(train_local_indices, train_labels_local, batch_size=batch_size)

    train_loader_loc = DataLoader(train_ds_loc, batch_sampler=train_sampler,
                                  num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, persistent_workers=False)
    val_loader_loc = DataLoader(val_ds_loc, batch_size=batch_size, shuffle=False,
                                num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, persistent_workers=False)
    return train_loader_loc, val_loader_loc, train_items_loc, val_items_loc

def build_plain_loader_from_indices(idx_global, batch_size=BATCH_SIZE):
    ds = Dataset([items[i] for i in idx_global], transform=val_tf)
    loader = DataLoader(ds, batch_size=batch_size, shuffle=False,
                        num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, persistent_workers=False)
    return loader, ds

# ==========================================
# Model (DenseNet with configurable dropout)
# ==========================================
from monai.networks.nets import DenseNet

class DenseNet3DHead(nn.Module):
    def __init__(self, out_ch=2, dropout_prob=0.0):
        super().__init__()
        self.backbone = DenseNet(
            spatial_dims=3, in_channels=1, out_channels=out_ch,
            init_features=64, growth_rate=32, block_config=(6,12,24,16),
            bn_size=4, act=("relu", {"inplace": False}),
            dropout_prob=dropout_prob
        )
    def forward(self, x): return self.backbone(x)

def build_binary_model(dropout_prob=0.0):
    model = DenseNet3DHead(out_ch=2, dropout_prob=dropout_prob).to(device)
    for m in model.modules():
        if isinstance(m, nn.ReLU): m.inplace = False
    return model

def unfreeze_all(model):
    for p in model.parameters(): p.requires_grad_(True)

def freeze_backbone_keep_classifier(model):
    """Freeze everything except the classifier head (MONAI DenseNet .backbone.class_layers.*)."""
    for n,p in model.named_parameters():
        p.requires_grad_(False)
    for n,p in model.named_parameters():
        if "backbone.class_layers" in n or "classifier" in n:
            p.requires_grad_(True)
    # sanity print
    n_tr = sum(p.numel() for p in model.parameters() if p.requires_grad)
    n_all = sum(p.numel() for p in model.parameters())
    print(f"FT trainable params: {n_tr}/{n_all} ({100*n_tr/n_all:.2f}%)")

class ModelEMA:
    def __init__(self, model, decay=0.999):
        import copy
        self.ema = copy.deepcopy(model).to(device)
        for p in self.ema.parameters(): p.requires_grad_(False)
        self.decay = decay
    @torch.no_grad()
    def update(self, model):
        d=self.decay; msd=model.state_dict()
        for k,v in self.ema.state_dict().items():
            if v.dtype.is_floating_point: v.copy_(v*d + msd[k]*(1-d))

def make_optim_sched(model, epochs=80, base_lr=5e-5, weight_decay=1e-5, warmup=5):
    opt = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()),
                      lr=base_lr, weight_decay=weight_decay)
    def lr_lambda(e):
        if e < warmup: return (e+1)/max(1,warmup)
        prog = (e-warmup)/max(1,(epochs-warmup))
        return 0.5*(1+math.cos(math.pi*min(prog,1.0)))
    sch = optim.lr_scheduler.LambdaLR(opt, lr_lambda=lr_lambda)
    return opt, sch

# MixUp (optional)
def mixup(x, y_ad, alpha=0.2):
    if alpha <= 0: return x, y_ad
    lam = np.random.beta(alpha, alpha)
    perm = torch.randperm(x.size(0), device=x.device)
    x_mix = lam * x + (1-lam) * x[perm]
    y_mix = lam * y_ad + (1-lam) * y_ad[perm]
    return x_mix, y_mix

# ----- TTA & single-logit helpers -----
def tta_logits(x, model_eval):
    outs=[]
    with torch.no_grad(), torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
        outs.append(model_eval(x))
        for dims in [(2,),(3,),(4,)]:
            outs.append(model_eval(torch.flip(x,dims=dims)))
        for a,b in [(2,3),(2,4),(3,4)]:
            for k in (1,2):
                outs.append(model_eval(torch.rot90(x,k=k,dims=(a,b))))
    return torch.stack(outs,0).mean(0)  # [N,2]

def two_to_single_logit(logits2):  # AD vs CN margin
    return logits2[:,0] - logits2[:,1]

@torch.no_grad()
def debug_val_bias_single(model_eval, loader):
    model_eval.eval()
    s=0.0; n=0
    for batch in loader:
        x = batch["image"].to(device, dtype=torch.float32)
        p_ad = torch.sigmoid(two_to_single_logit(tta_logits(x, model_eval)))
        s += p_ad.sum().item(); n += p_ad.numel()
    avg_ad = s/max(1,n)
    print(f"Val avg P(AD) ~= {avg_ad:.4f} | P(CN) ~= {1-avg_ad:.4f}")

@torch.no_grad()
def evaluate_single(model_eval, loader, sweep_thresholds=True, print_quantiles=True):
    model_eval.eval()
    p_ad_all, y_all = [], []
    for batch in loader:
        x = batch["image"].to(device, dtype=torch.float32, non_blocking=True)
        y = batch["label"].long().to(device, non_blocking=True)  # 0=AD, 1=CN
        p_ad = torch.sigmoid(two_to_single_logit(tta_logits(x, model_eval)))
        p_ad_all.append(p_ad.cpu().numpy()); y_all.append(y.cpu().numpy())
    p_ad = np.concatenate(p_ad_all); y = np.concatenate(y_all)

    preds = np.where(p_ad>=0.5, 0, 1)
    acc = accuracy_score(y, preds)
    f1  = f1_score(y, preds)
    auc = roc_auc_score(y, 1.0 - p_ad)

    print("\n=== Eval (BCE single-logit, th=0.50) ===")
    print(f"Acc:{acc:.4f} | F1(CN+):{f1:.4f} | AUC(CN prob):{auc:.4f}")
    print("Confusion:\n", confusion_matrix(y, preds))
    print(classification_report(y, preds, target_names=["AD","CN"]))
    if print_quantiles:
        qs = np.quantile(p_ad, [0.05,0.25,0.5,0.75,0.95]); print("P(AD) q=[5,25,50,75,95]:", np.round(qs,3))

    if sweep_thresholds:
        best, best_t = -1, 0.5
        for t in np.linspace(0.05,0.95,37):
            preds_t = np.where(p_ad>=t, 0, 1)
            f1_t = f1_score(y, preds_t)
            if f1_t > best: best, best_t = f1_t, t
        preds_best = np.where(p_ad>=best_t, 0, 1)
        print(f"--- Threshold sweep on P(AD) ---\nBest F1(CN+)={best:.4f} at th={best_t:.2f}")
        print(confusion_matrix(y, preds_best))
    return acc, f1, auc

def compute_balanced_accuracy(cm):
    tp_ad = cm[0,0]; fn_ad = cm[0,1]
    tn_ad = cm[1,1]; fp_ad = cm[1,0]
    tpr = tp_ad / max(1,(tp_ad+fn_ad))
    tnr = tn_ad / max(1,(tn_ad+fp_ad))
    return 0.5*(tpr+tnr)

@torch.no_grad()
def evaluate_with_threshold(model_eval, loader, threshold: float, title="FINAL"):
    """Evaluate at a chosen P(AD) threshold; prints Confusion + Balanced Accuracy."""
    model_eval.eval()
    p_ad_all, y_all = [], []
    for batch in loader:
        x = batch["image"].to(device, dtype=torch.float32, non_blocking=True)
        y = batch["label"].long().to(device, non_blocking=True)  # 0=AD, 1=CN
        p_ad = torch.sigmoid(two_to_single_logit(tta_logits(x, model_eval)))
        p_ad_all.append(p_ad.cpu().numpy()); y_all.append(y.cpu().numpy())
    p_ad = np.concatenate(p_ad_all); y = np.concatenate(y_all)

    preds = np.where(p_ad>=threshold, 0, 1)
    acc = accuracy_score(y, preds)
    cm  = confusion_matrix(y, preds, labels=[0,1])
    balacc = compute_balanced_accuracy(cm)

    print(f"\n=== {title} @ th={threshold:.2f} ===")
    print(f"Accuracy: {acc:.4f} | Balanced Acc: {balacc:.4f}")
    print("Confusion (rows=GT [AD,CN], cols=Pred [AD,CN]):\n", cm)
    print(classification_report(y, preds, target_names=["AD","CN"]))
    return acc, balacc, cm

# Threshold search: best Overall Accuracy and best Balanced Accuracy
def sweep_thresholds_all(p_ad, y, grid=np.linspace(0.05, 0.95, 37)):
    best = {
        "acc":(-1,None), "balacc":(-1,None), "f1_ad":(-1,None), "f1_cn":(-1,None)
    }
    for t in grid:
        preds = np.where(p_ad>=t, 0, 1)
        acc = accuracy_score(y, preds)
        cm  = confusion_matrix(y, preds, labels=[0,1])
        balacc = compute_balanced_accuracy(cm)
        from sklearn.metrics import f1_score
        f1_ad = f1_score(y==0, preds==0)
        f1_cn = f1_score(y==1, preds==1)
        if acc    > best["acc"][0]   : best["acc"]    = (acc, t)
        if balacc > best["balacc"][0]: best["balacc"] = (balacc, t)
        if f1_ad  > best["f1_ad"][0] : best["f1_ad"]  = (f1_ad, t)
        if f1_cn  > best["f1_cn"][0] : best["f1_cn"]  = (f1_cn, t)
    return best

@torch.no_grad()
def collect_p_ad_and_labels(model_eval, loader):
    p_ad_all, y_all = [], []
    for batch in loader:
        x = batch["image"].to(device, dtype=torch.float32, non_blocking=True)
        y = batch["label"].long().cpu().numpy()
        logits2 = tta_logits(x, model_eval)
        p_ad = torch.sigmoid(two_to_single_logit(logits2)).cpu().numpy()
        p_ad_all.append(p_ad); y_all.append(y)
    return np.concatenate(p_ad_all), np.concatenate(y_all)

# ================================================================
# Train loop (supports resume & configurable dropout / weighting)
# ================================================================
def train_binary_fold_balanced(idx_train, idx_val,
                               epochs=80,
                               base_lr=5e-5,
                               ema_decay=0.999,
                               batch_size=BATCH_SIZE,
                               grad_accum_target=8,
                               mixup_alpha=0.2,
                               use_class_weight=True,
                               pos_weight_power=1.25,
                               patience=12,
                               dropout_prob=0.0,
                               resume_path=None,
                               freeze_backbone=False,
                               save_name="best_binary_ad_cn_ema.pth"):

    # Build loaders (balanced over LOCAL indices)
    train_loader, val_loader, train_items_fold, _ = build_balanced_loaders_from_indices(
        idx_train, idx_val, batch_size=batch_size
    )

    # BCEWithLogits: AD positive (label 0)
    y_train = np.array([it["label"] for it in train_items_fold])  # 0=AD,1=CN
    n_ad = (y_train==0).sum(); n_cn=(y_train==1).sum()
    ratio = n_cn / max(1, n_ad)
    pos_weight_val = max(1.0, ratio**pos_weight_power)

    print(f"\nTrain size: {len(train_items_fold)} | Val size: {len(val_loader.dataset)}")
    print(f"Batch size: {batch_size} (grad accum to ~{grad_accum_target})")
    print(f"Dropout in DenseNet blocks: {dropout_prob}")
    print(f"Class ratio (CN/AD) ~ {ratio:.3f} -> pos_weight(AD) base={pos_weight_val:.3f} (if enabled)")

    model = build_binary_model(dropout_prob=dropout_prob); unfreeze_all(model)

    # Optional resume (useful for FT with new dropout)
    if resume_path is not None and Path(resume_path).exists():
        ckpt = torch.load(resume_path, map_location=device, weights_only=False)
        missing, unexpected = model.load_state_dict(ckpt["model"], strict=False)
        print(f"Resumed from: {resume_path}\n  Missing keys: {len(missing)} | Unexpected: {len(unexpected)}")

    # Optional freezing (FT only)
    if freeze_backbone:
        freeze_backbone_keep_classifier(model)

    # Build loss
    if use_class_weight:
        pos_weight = torch.tensor([pos_weight_val], device=device, dtype=torch.float32)
        bce = nn.BCEWithLogitsLoss(pos_weight=pos_weight).to(device)
        print(f"Using class weight: pos_weight(AD)={pos_weight.item():.3f}")
    else:
        bce = nn.BCEWithLogitsLoss().to(device)
        print("Using unweighted BCE")

    opt, sch = make_optim_sched(model, epochs=epochs, base_lr=base_lr)
    ema = ModelEMA(model, decay=ema_decay)
    scaler = torch.amp.GradScaler('cuda', enabled=torch.cuda.is_available())

    best_key=-1; best_path=SAVE_DIR/save_name; no_improve=0
    accum_steps = max(1, grad_accum_target // batch_size)
    print(f"Grad accumulation steps: {accum_steps}")

    for epoch in range(1, epochs+1):
        model.train(); running=0.0; opt.zero_grad(set_to_none=True)
        print(f"\n--- Epoch {epoch}/{epochs} ---")
        pbar = tqdm(train_loader, desc="Train", leave=True, mininterval=0.2, dynamic_ncols=True)
        step_i = 0

        for batch in pbar:
            x = batch["image"].to(device, dtype=torch.float32, non_blocking=True)
            y = batch["label"].long().to(device, non_blocking=True)   # 0=AD,1=CN
            y_ad = (y==0).float().unsqueeze(1)                        # AD-positive

            # Optional MixUp
            if mixup_alpha > 0:
                x, y_ad = mixup(x, y_ad, alpha=mixup_alpha)

            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                logits2 = model(x)                    # [N,2]
                single = two_to_single_logit(logits2) # [N]
                loss   = bce(single.unsqueeze(1), y_ad) / accum_steps

            scaler.scale(loss).backward()
            step_i += 1
            if step_i % accum_steps == 0:
                scaler.unscale_(opt)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(opt); scaler.update()
                opt.zero_grad(set_to_none=True)
                ema.update(model)

            running += float(loss.detach().cpu()) * x.size(0) * accum_steps
            pbar.set_postfix(loss=float(loss.detach().cpu())*accum_steps)

        sch.step()
        train_loss = running / len(train_loader.dataset)
        print(f"Train loss: {train_loss:.4f}")

        # Validation snapshot
        debug_val_bias_single(ema.ema, val_loader)
        acc,f1,auc = evaluate_single(ema.ema, val_loader, sweep_thresholds=True, print_quantiles=True)

        # Composite key (favor AUC, then F1, then Acc)
        key = (0.0 if np.isnan(auc) else auc) + 0.01*f1 + 0.001*acc
        print(f"Score key: {key:.4f}")

        if key>best_key:
            best_key=key; no_improve=0
            torch.save({"model":ema.ema.state_dict(),"epoch":epoch,
                        "val_acc":float(acc),"val_f1":float(f1),"val_auc":float(auc)}, best_path)
            print("✅ Saved new best.")
        else:
            no_improve+=1
            if no_improve>=patience:
                print("⏹️ Early stopping."); break

    # Final eval (reload best)
    ckpt=torch.load(best_path, map_location=device, weights_only=False)
    ema.ema.load_state_dict(ckpt["model"])
    print("\n=== BEST (reloaded) ===")
    evaluate_single(ema.ema, val_loader, sweep_thresholds=True)
    print("Model saved to:", best_path)
    return best_path

# ===========================
# Split: train / val / test
# ===========================
labels = np.array([it["label"] for it in items])
idx_all = np.arange(len(items))

# held-out TEST
idx_trainval, idx_test = train_test_split(idx_all, test_size=TEST_FRACTION, stratify=labels, random_state=SEED)
labels_trainval = labels[idx_trainval]
# train/val
idx_train, idx_val = train_test_split(idx_trainval, test_size=VAL_FRACTION_OF_TRAINVAL, stratify=labels_trainval, random_state=SEED)

print(f"\nSplit sizes -> Train: {len(idx_train)} | Val: {len(idx_val)} | Test: {len(idx_test)}")

# ===========================
# Phase-1 train (baseline)
# ===========================
PHASE1_CKPT = SAVE_DIR/"best_binary_ad_cn_ema.pth"
PHASE2_CKPT = SAVE_DIR/"best_binary_ad_cn_ema_ft.pth"

best_p1_path = train_binary_fold_balanced(
    idx_train, idx_val,
    epochs=EPOCHS_PHASE1,
    base_lr=LR_PHASE1,
    ema_decay=0.999,
    batch_size=BATCH_SIZE,
    grad_accum_target=GRAD_ACCUM_TARGET,
    mixup_alpha=MIXUP_ALPHA_P1,
    use_class_weight=USE_CLASS_WEIGHT_P1,
    pos_weight_power=POS_WEIGHT_POWER,
    patience=PATIENCE_P1,
    dropout_prob=DROPOUT_P1,
    resume_path=None,
    freeze_backbone=False,
    save_name=PHASE1_CKPT.name
)

# =====================================
# Threshold selection on VAL (Phase-1)
# =====================================
_, val_loader, _, _ = build_balanced_loaders_from_indices(idx_train, idx_val, batch_size=BATCH_SIZE)
model_p1 = build_binary_model(dropout_prob=DROPOUT_P1)
ckpt = torch.load(best_p1_path, map_location=device, weights_only=False)
model_p1.load_state_dict(ckpt["model"]); model_p1.eval()

p_ad_val_p1, y_val_p1 = collect_p_ad_and_labels(model_p1, val_loader)
best_p1 = sweep_thresholds_all(p_ad_val_p1, y_val_p1)
print("\nThreshold suggestions (VALIDATION, Phase-1):")
print(f"- Max Accuracy     -> t={best_p1['acc'][1]:.2f}, Acc={best_p1['acc'][0]:.3f}")
print(f"- Balanced Acc     -> t={best_p1['balacc'][1]:.2f}, BalAcc={best_p1['balacc'][0]:.3f}")

with open(SAVE_DIR/"deploy_threshold_phase1.json","w") as f:
    json.dump({"t_acc": float(best_p1['acc'][1]), "t_balacc": float(best_p1['balacc'][1])}, f)
print("Saved Phase-1 thresholds:", SAVE_DIR/"deploy_threshold_phase1.json")

# ================================================
# FINAL TEST (Phase-1) — evaluate both thresholds
# ================================================
test_loader, _ = build_plain_loader_from_indices(idx_test, batch_size=BATCH_SIZE)
evaluate_with_threshold(model_p1, test_loader, threshold=float(best_p1['acc'][1]), title="FINAL TEST (Phase-1, Max-Acc)")
evaluate_with_threshold(model_p1, test_loader, threshold=float(best_p1['balacc'][1]), title="FINAL TEST (Phase-1, Max-BalAcc)")

# ===========================
# Phase-2 Fine-Tune (gentle)
# ===========================
best_p2_path = train_binary_fold_balanced(
    idx_train, idx_val,
    epochs=EPOCHS_PHASE2,
    base_lr=LR_PHASE2,
    ema_decay=0.999,
    batch_size=BATCH_SIZE,
    grad_accum_target=GRAD_ACCUM_TARGET,
    mixup_alpha=MIXUP_ALPHA_P2,            # OFF in FT
    use_class_weight=USE_CLASS_WEIGHT_P2,  # OFF in FT
    pos_weight_power=POS_WEIGHT_POWER,     # ignored if not using class weight
    patience=PATIENCE_P2,
    dropout_prob=DROPOUT_P2,
    resume_path=best_p1_path,              # start from Phase-1 weights
    freeze_backbone=FREEZE_BACKBONE_IN_FT, # optionally freeze backbone
    save_name=PHASE2_CKPT.name
)

# Threshold selection on VAL (Phase-2) for BOTH metrics
model_p2 = build_binary_model(dropout_prob=DROPOUT_P2)
ckpt_ft = torch.load(best_p2_path, map_location=device, weights_only=False)
model_p2.load_state_dict(ckpt_ft["model"]); model_p2.eval()

p_ad_val_p2, y_val_p2 = collect_p_ad_and_labels(model_p2, val_loader)
best_p2 = sweep_thresholds_all(p_ad_val_p2, y_val_p2)
print("\nThreshold suggestions (VALIDATION, Phase-2/Fine-Tune):")
print(f"- Max Accuracy     -> t={best_p2['acc'][1]:.2f}, Acc={best_p2['acc'][0]:.3f}")
print(f"- Balanced Acc     -> t={best_p2['balacc'][1]:.2f}, BalAcc={best_p2['balacc'][0]:.3f}")

with open(SAVE_DIR/"deploy_threshold_phase2_ft.json","w") as f:
    json.dump({"t_acc": float(best_p2['acc'][1]), "t_balacc": float(best_p2['balacc'][1])}, f)
print("Saved Phase-2 thresholds:", SAVE_DIR/"deploy_threshold_phase2_ft.json")

# FINAL TEST (Phase-2) — evaluate both thresholds
evaluate_with_threshold(model_p2, test_loader, threshold=float(best_p2['acc'][1]), title="FINAL TEST (Phase-2, Max-Acc)")
evaluate_with_threshold(model_p2, test_loader, threshold=float(best_p2['balacc'][1]), title="FINAL TEST (Phase-2, Max-BalAcc)")

# ================================================
# Ensemble Phase-1 + Phase-2 (often best of both)
# ================================================
@torch.no_grad()
def evaluate_ensemble(models, loader, threshold, title="FINAL TEST (Ensemble)"):
    for m in models: m.eval()
    p_ad_all, y_all = [], []
    for batch in loader:
        x = batch["image"].to(device, dtype=torch.float32, non_blocking=True)
        y = batch["label"].long().cpu().numpy()
        # average logits across models with TTA
        logits = []
        for m in models:
            logits.append(tta_logits(x, m))
        logits = torch.stack(logits, 0).mean(0)
        p_ad = torch.sigmoid(two_to_single_logit(logits)).cpu().numpy()
        p_ad_all.append(p_ad); y_all.append(y)
    p_ad = np.concatenate(p_ad_all); y = np.concatenate(y_all)
    preds = np.where(p_ad>=threshold, 0, 1)
    acc = accuracy_score(y, preds)
    cm  = confusion_matrix(y, preds, labels=[0,1])
    balacc = compute_balanced_accuracy(cm)
    print(f"\n=== {title} @ th={threshold:.2f} ===")
    print(f"Accuracy: {acc:.4f} | Balanced Acc: {balacc:.4f}")
    print("Confusion (rows=GT [AD,CN], cols=Pred [AD,CN]):\n", cm)
    print(classification_report(y, preds, target_names=['AD','CN']))
    return acc, balacc, cm

# Build eval models
m1 = build_binary_model(dropout_prob=DROPOUT_P1)
m1.load_state_dict(torch.load(PHASE1_CKPT, map_location=device, weights_only=False)["model"]); m1.eval()
m2 = build_binary_model(dropout_prob=DROPOUT_P2)
m2.load_state_dict(torch.load(PHASE2_CKPT, map_location=device, weights_only=False)["model"]); m2.eval()

# Ensemble thresholds (sweep on VAL using ensemble)
@torch.no_grad()
def ensemble_val_thresholds(m1, m2, loader):
    p_ad_all, y_all = [], []
    for batch in loader:
        x = batch["image"].to(device, dtype=torch.float32, non_blocking=True)
        y = batch["label"].long().cpu().numpy()
        log1 = tta_logits(x, m1); log2 = tta_logits(x, m2)
        logits = (log1 + log2)/2
        p_ad = torch.sigmoid(two_to_single_logit(logits)).cpu().numpy()
        p_ad_all.append(p_ad); y_all.append(y)
    p_ad = np.concatenate(p_ad_all); y = np.concatenate(y_all)
    return sweep_thresholds_all(p_ad, y)

best_ens = ensemble_val_thresholds(m1, m2, val_loader)
print("\nThreshold suggestions (VALIDATION, Ensemble P1+P2):")
print(f"- Max Accuracy     -> t={best_ens['acc'][1]:.2f}, Acc={best_ens['acc'][0]:.3f}")
print(f"- Balanced Acc     -> t={best_ens['balacc'][1]:.2f}, BalAcc={best_ens['balacc'][0]:.3f}")

# FINAL TEST (Ensemble) — evaluate both thresholds
evaluate_ensemble([m1, m2], test_loader, threshold=float(best_ens['acc'][1]), title="FINAL TEST (Ensemble, Max-Acc)")
evaluate_ensemble([m1, m2], test_loader, threshold=float(best_ens['balacc'][1]), title="FINAL TEST (Ensemble, Max-BalAcc)")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.6/766.6 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Train: 100%|██████████| 291/291 [03:10<00:00,  1.53it/s, loss=0.971]

Train loss: 1.2265


Val avg P(AD) ~= 0.4948 | P(CN) ~= 0.5052

=== Eval (BCE single-logit, th=0.50) ===
Acc:0.6621 | F1(CN+):0.7967 | AUC(CN prob):0.3594
Confusion:
 [[  0  74]
 [  0 145]]
              precision    recall  f1-score   support

          AD       0.00      0.00      0.00        74
          CN       0.66      1.00      0.80       145

    accuracy                           0.66       219
   macro avg       0.33      0.50      0.40       219
weighted avg       0.44      0.66      0.53       219

P(AD) q=[5,25,50,75,95]: [0.493 0.494 0.495 0.496 0.496]
--- Threshold sweep on P(AD) ---
Best F1(CN+)=0.7967 at th=0.50
[[  0  74]
 [  0 145]]
Score key: 0.3680
✅ Saved new best.

--- Epoch 2/80 ---


Train: 100%|██████████| 291/291 [02:27<00:00,  1.98it/s, loss=0.732]

...

Train loss: 0.6603


Val avg P(AD) ~= 0.5087 | P(CN) ~= 0.4913

=== Eval (BCE single-logit, th=0.50) ===
Acc:0.7397 | F1(CN+):0.7799 | AUC(CN prob):0.8613
Confusion:
 [[ 61  13]
 [ 44 101]]
              precision    recall  f1-score   support

          AD       0.58      0.82      0.68        74
          CN       0.89      0.70      0.78       145

    accuracy                           0.74       219
   macro avg       0.73      0.76      0.73       219
weighted avg       0.78      0.74      0.75       219

P(AD) q=[5,25,50,75,95]: [0.058 0.125 0.458 0.929 0.979]
--- Threshold sweep on P(AD) ---
Best F1(CN+)=0.8656 at th=0.92
[[ 46  28]
 [ 13 132]]
Score key: 0.8699

--- Epoch 9/20 ---


Train: 100%|██████████| 291/291 [01:51<00:00,  2.61it/s, loss=0.0402]


Train loss: 0.6276
Val avg P(AD) ~= 0.4974 | P(CN) ~= 0.5026

=== Eval (BCE single-logit, th=0.50) ===
Acc:0.7489 | F1(CN+):0.7893 | AUC(CN prob):0.8603
Confusion:
 [[ 61  13]
 [ 42 103]]
              precision    recall  f1-score   support

          AD       0.59      0.82      0.69        74
          CN       0.89      0.71      0.79       145

    accuracy                           0.75       219
   macro avg       0.74      0.77      0.74       219
weighted avg       0.79      0.75      0.76       219

P(AD) q=[5,25,50,75,95]: [0.053 0.109 0.423 0.926 0.98 ]
--- Threshold sweep on P(AD) ---
Best F1(CN+)=0.8644 at th=0.95
[[ 39  35]
 [  8 137]]
Score key: 0.8690
⏹️ Early stopping.

=== BEST (reloaded) ===

=== Eval (BCE single-logit, th=0.50) ===
Acc:0.8128 | F1(CN+):0.8520 | AUC(CN prob):0.8763
Confusion:
 [[ 60  14]
 [ 27 118]]
              precision    recall  f1-score   support

          AD       0.69      0.81      0.75        74
          CN       0.89      0.81      0.85

(0.8350515463917526,
 np.float64(0.8416219439475254),
 array([[ 56,   9],
        [ 23, 106]]))